# Actividad 3 - Métodos de aprendizaje supervisado

Cindy Johanna Zapata Romero

# Paso 1: Crear un Dataset Simulado
Usaré Python y pandas para crear un dataset que se puede usar en el desarrollo de un modelo de aprendizaje supervisado.

In [25]:
import pandas as pd
import numpy as np

# Creamos los datos simulados para el sistema de transporte masivo
np.random.seed(42)

# Definimos las estaciones de ejemplo
estaciones = ['A', 'B', 'C', 'D', 'E', 'F']

# Creamos un dataframe simulando viajes
data = {
    'origen': np.random.choice(estaciones, 100),
    'destino': np.random.choice(estaciones, 100),
    'tiempo_viaje': np.random.randint(5, 60, size=100),  # Tiempo de viaje entre 5 y 60 minutos
    'num_pasajeros': np.random.randint(10, 300, size=100),  # Número de pasajeros entre 10 y 300
    'clima': np.random.choice(['soleado', 'lluvioso', 'nublado'], 100),  # Condición climática
    'retraso': np.random.choice([0, 1], 100, p=[0.8, 0.2])  # 0 = sin retraso, 1 = con retraso
}

df = pd.DataFrame(data)

# Mostramos las primeras filas del dataset
df.head()

,origen,destino,tiempo_viaje,num_pasajeros,clima,retraso
0,D,E,36,217,nublado,1
1,E,A,36,140,lluvioso,1
2,C,A,28,161,soleado,0
3,E,A,45,63,lluvioso,1
4,E,A,56,129,lluvioso,0


# Paso 2: Preparar los Datos para el Modelo de Aprendizaje Supervisado
El objetivo sea predecir si un viaje tendrá retraso o no, esa sería la variable objetivo (retraso). Por lo tanto vamos a preparar los datos para entrenar un modelo de clasificación.

In [27]:
from sklearn.preprocessing import StandardScaler

# Convertimos las columnas categóricas a variables dummy
df = pd.get_dummies(df, columns=['origen', 'destino', 'clima'], drop_first=True)

# Escalamos características numéricas
scaler = StandardScaler()
df[['tiempo_viaje', 'num_pasajeros']] = scaler.fit_transform(df[['tiempo_viaje', 'num_pasajeros']])

# Separamos las variables predictoras y la variable objetivo
X = df.drop('retraso', axis=1)  # Variables predictoras
y = df['retraso']  # Variable objetivo

# Dividimos el conjunto de datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Mostramos la estructura de los datos de entrenamiento en pantalla
X_train.head()

,tiempo_viaje,num_pasajeros,origen_B,origen_C,origen_D,origen_E,origen_F,destino_B,destino_C,destino_D,destino_E,destino_F,clima_nublado,clima_soleado
55,-0.459152,-1.358174,False,False,False,True,False,False,False,False,True,False,False,True
88,-1.590654,-1.370089,False,False,False,True,False,False,True,False,False,False,False,False
26,-0.518704,-1.405834,False,False,True,False,False,False,False,False,False,True,False,False
42,1.446536,-0.047540,False,False,False,True,False,False,False,True,False,False,False,True
69,-0.578257,0.857989,False,False,False,False,True,False,False,True,False,False,False,False


# Paso 3: Entrenar un Modelo de Aprendizaje Supervisado (Clasificación)
Voy a utilizar un modelo de clasificación, decidí usar en este caso, un árbol de decisión para predecir si un viaje tendrá retraso.

## Módelo 1: Árbol de decisión

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Definimos el modelo del árbol de decisión
modelo = DecisionTreeClassifier(random_state=42)

# Definimos los hiperparámetros que queremos probar
parametros = {
    'max_depth': [2, 5, 10, 20],
    'min_samples_leaf': [1, 5, 10],
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']  # Añadir más criterios para probar
}

# Aplicamos GridSearchCV para encontrar la mejor combinación de hiperparámetros
grid_search = GridSearchCV(modelo, parametros, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Mostramos los mejores parámetros encontrados
print(f"Mejores hiperparámetros: {grid_search.best_params_}")

# Evaluamos el modelo con los mejores parámetros
mejor_modelo = grid_search.best_estimator_
y_pred = mejor_modelo.predict(X_test)

# Mostramos la nueva exactitud
print(f"Exactitud del modelo optimizado: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print(classification_report(y_test, y_pred))

Mejores hiperparámetros: {'criterion': 'gini', 'max_depth': 2, 'min_samples_leaf': 1, 'splitter': 'random'}
Exactitud del modelo optimizado: 55.00%
              precision    recall  f1-score   support

           0       0.59      0.83      0.69        12
           1       0.33      0.12      0.18         8

    accuracy                           0.55        20
   macro avg       0.46      0.48      0.44        20
weighted avg       0.49      0.55      0.49        20



## Modelo 2: Random Forest
El modelo Random Forest es un conjunto de árboles de decisión, lo que suele mejorar la precisión y reducir el sobreajuste.

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Definimos el modelo Random Forest
rf_model = RandomForestClassifier(random_state=42)

# Entrenamos el modelo
rf_model.fit(X_train, y_train)

# Hacemos predicciones
y_pred_rf = rf_model.predict(X_test)

# Evaluamos el modelo
print(f"Exactitud del modelo Random Forest: {accuracy_score(y_test, y_pred_rf) * 100:.2f}%")
print(classification_report(y_test, y_pred_rf))

Exactitud del modelo Random Forest: 60.00%
              precision    recall  f1-score   support

           0       0.61      0.92      0.73        12
           1       0.50      0.12      0.20         8

    accuracy                           0.60        20
   macro avg       0.56      0.52      0.47        20
weighted avg       0.57      0.60      0.52        20



# Modelo 3: SVM (Support Vector Machine)
SVM es otro algoritmo para clasificación que intenta encontrar el mejor hiperplano que separe las clases de datos.

In [35]:
from sklearn.svm import SVC

# Definimos el modelo SVM
svm_model = SVC(random_state=42)

# Entrenamos el modelo
svm_model.fit(X_train, y_train)

# Hacemos predicciones
y_pred_svm = svm_model.predict(X_test)

# Evaluamos el modelo
print(f"Exactitud del modelo SVM: {accuracy_score(y_test, y_pred_svm) * 100:.2f}%")
print(classification_report(y_test, y_pred_svm))

Exactitud del modelo SVM: 60.00%
              precision    recall  f1-score   support

           0       0.61      0.92      0.73        12
           1       0.50      0.12      0.20         8

    accuracy                           0.60        20
   macro avg       0.56      0.52      0.47        20
weighted avg       0.57      0.60      0.52        20



# Gracias